In [1]:
import numpy as np
from keras.datasets import mnist

import keras
from keras.layers import Input, UpSampling2D, Conv2DTranspose, Conv2D, LeakyReLU
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop
from tensorflow.examples.tutorials.mnist import input_data
from keras.layers.normalization import *
import matplotlib.pyplot as plt
%matplotlib inline


Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
x_train = input_data.read_data_sets("mnist",one_hot=True).train.images
x_train = x_train.reshape(-1, 28,28, 1).astype(np.float32)

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Extracting mnist\train-images-idx3-ubyte.gz
Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
60000 train samples
10000 test samples


In [4]:
#Build discriminator
D = Sequential()
depth = 64
dropout = 0.4

input_shape = (28,28,1)

#output 14 x 14 x 64
D.add(Conv2D(64, 5, strides = 2, input_shape = input_shape, padding='same', activation=LeakyReLU(alpha=0.2)))
D.add(Dropout(dropout))

#output 7 x 7 x 128
D.add(Conv2D(128, 5, strides = 2, input_shape = input_shape, padding='same', activation=LeakyReLU(alpha=0.2)))
D.add(Dropout(dropout))

#output 4 x 4 x 256
D.add(Conv2D(256, 5, strides = 2, input_shape = input_shape, padding='same', activation=LeakyReLU(alpha=0.2)))
D.add(Dropout(dropout))

#output 4 x 4 x 512
D.add(Conv2D(512, 5, strides = 1, input_shape = input_shape, padding='same', activation=LeakyReLU(alpha=0.2)))
D.add(Dropout(dropout))

# Out: 1-dim probability

D.add(Flatten())
D.add(Dense(1))

D.add(Activation('sigmoid'))

D.summary()

C:\ProgramData\Anaconda3\envs\carnd-term1\lib\site-packages\keras\activations.py:89: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         819456    
_________________________________________________________________
dropout_3 (Dropout)          (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 512)         3277312   
__________

In [5]:
#Build generator
nch = 200
g_input = Input(shape=[100])
depth = 64+64+64+64
dim = 7

G = Sequential()

G.add(Dense(7*7*256, input_dim=100))
G.add(BatchNormalization(momentum=0.9))
G.add(Activation('relu'))
G.add(Reshape((7, 7, 256)))
G.add(Dropout(0.4))
            
# Input 7 x 7 x 256
# Output 14 x 14 x 128

G.add(UpSampling2D())
G.add(Conv2DTranspose(int(128), 5, padding='same'))
G.add(BatchNormalization(momentum=0.9))
G.add(Activation('relu'))

# Input 14 x 14 x 128
# Output 28 x 28 x 64

G.add(UpSampling2D())
G.add( Conv2DTranspose(int(64), 5, padding='same'))
G.add(BatchNormalization(momentum=0.9))
G.add(Activation ( 'relu'))

# Input 28 x 28 x 64
# Output 28 x 28 x 32

G.add( Conv2DTranspose(int(32), 5, padding='same'))
G.add( BatchNormalization(momentum=0.9))
G.add( Activation('relu'))

# Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
G.add( Conv2DTranspose(1, 5, padding='same'))
G.add( Activation('sigmoid'))

G.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_1 (Batch (None, 12544)             50176     
_________________________________________________________________
activation_2 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 128)       819328    
__________

In [6]:
# Discriminator model
optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
DM = Sequential()
DM.add(D)
DM.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [7]:
# Adversarial model
optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
AM = Sequential()
AM.add(G)
AM.add(D)
AM.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

In [8]:
train_steps=10000 
batch_size=256 
save_interval=0

noise_input = None
if save_interval>0:
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])

for i in range(train_steps):
    images_train = x_train[np.random.randint(0,x_train.shape[0], size=batch_size), :, :, :]
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
    images_fake = G.predict(noise)
    #print(images_fake.shape)
    #print(images_train.shape)
    x = np.concatenate((images_train, images_fake))
    y = np.ones([2*batch_size, 1])
    y[batch_size:, :] = 0
    d_loss = DM.train_on_batch(x, y)
        
    y = np.ones([batch_size, 1])
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
    a_loss = AM.train_on_batch(noise, y)
    log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
    log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
    print(log_mesg)

C:\ProgramData\Anaconda3\envs\carnd-term1\lib\site-packages\keras\backend\tensorflow_backend.py:2289: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


0: [D loss: 0.694201, acc: 0.490234]  [A loss: 2.335869, acc: 0.000000]
1: [D loss: 1.288092, acc: 0.500000]  [A loss: 0.979445, acc: 0.007812]
2: [D loss: 1.829371, acc: 0.500000]  [A loss: 0.648161, acc: 0.808594]
3: [D loss: 0.634075, acc: 0.500000]  [A loss: 0.458573, acc: 1.000000]
4: [D loss: 0.401074, acc: 0.978516]  [A loss: 0.000939, acc: 1.000000]
5: [D loss: 0.134089, acc: 0.982422]  [A loss: 0.000030, acc: 1.000000]
6: [D loss: 0.035092, acc: 0.994141]  [A loss: 0.000002, acc: 1.000000]
7: [D loss: 0.018088, acc: 1.000000]  [A loss: 0.000003, acc: 1.000000]
8: [D loss: 0.010521, acc: 1.000000]  [A loss: 0.000007, acc: 1.000000]
9: [D loss: 0.011623, acc: 0.998047]  [A loss: 0.000000, acc: 1.000000]
10: [D loss: 0.008387, acc: 1.000000]  [A loss: 0.000000, acc: 1.000000]
11: [D loss: 0.007431, acc: 1.000000]  [A loss: 0.000000, acc: 1.000000]
12: [D loss: 0.006964, acc: 1.000000]  [A loss: 0.000000, acc: 1.000000]
13: [D loss: 0.004517, acc: 1.000000]  [A loss: 0.000000, acc

113: [D loss: 0.651379, acc: 0.710938]  [A loss: 0.715989, acc: 0.406250]
114: [D loss: 0.678389, acc: 0.484375]  [A loss: 0.980237, acc: 0.000000]
115: [D loss: 0.657582, acc: 0.585938]  [A loss: 0.749250, acc: 0.316406]
116: [D loss: 0.693254, acc: 0.482422]  [A loss: 1.234585, acc: 0.000000]
117: [D loss: 0.677019, acc: 0.576172]  [A loss: 0.727427, acc: 0.316406]
118: [D loss: 0.686112, acc: 0.494141]  [A loss: 0.738992, acc: 0.230469]
119: [D loss: 0.680883, acc: 0.478516]  [A loss: 0.755528, acc: 0.128906]
120: [D loss: 0.657070, acc: 0.535156]  [A loss: 0.753086, acc: 0.164062]
121: [D loss: 0.678862, acc: 0.507812]  [A loss: 0.822657, acc: 0.019531]
122: [D loss: 0.673938, acc: 0.531250]  [A loss: 0.754516, acc: 0.164062]
123: [D loss: 0.668311, acc: 0.501953]  [A loss: 0.868371, acc: 0.011719]
124: [D loss: 0.668536, acc: 0.542969]  [A loss: 0.895089, acc: 0.003906]
125: [D loss: 0.690350, acc: 0.490234]  [A loss: 0.827881, acc: 0.058594]
126: [D loss: 0.684987, acc: 0.494141]

224: [D loss: 0.697639, acc: 0.496094]  [A loss: 0.751786, acc: 0.054688]
225: [D loss: 0.692423, acc: 0.537109]  [A loss: 0.730889, acc: 0.183594]
226: [D loss: 0.699286, acc: 0.498047]  [A loss: 0.747506, acc: 0.089844]
227: [D loss: 0.688348, acc: 0.541016]  [A loss: 0.710334, acc: 0.371094]
228: [D loss: 0.723094, acc: 0.490234]  [A loss: 0.748376, acc: 0.156250]
229: [D loss: 0.719412, acc: 0.498047]  [A loss: 0.873381, acc: 0.000000]
230: [D loss: 0.692180, acc: 0.500000]  [A loss: 0.736092, acc: 0.199219]
231: [D loss: 0.699805, acc: 0.464844]  [A loss: 0.764583, acc: 0.031250]
232: [D loss: 0.691190, acc: 0.513672]  [A loss: 0.696989, acc: 0.441406]
233: [D loss: 0.695963, acc: 0.496094]  [A loss: 0.711659, acc: 0.335938]
234: [D loss: 0.702899, acc: 0.488281]  [A loss: 0.734365, acc: 0.089844]
235: [D loss: 0.692360, acc: 0.503906]  [A loss: 0.703960, acc: 0.324219]
236: [D loss: 0.694030, acc: 0.498047]  [A loss: 0.716009, acc: 0.273438]
237: [D loss: 0.697627, acc: 0.498047]

335: [D loss: 0.702790, acc: 0.513672]  [A loss: 0.848111, acc: 0.035156]
336: [D loss: 0.689752, acc: 0.558594]  [A loss: 0.755152, acc: 0.187500]
337: [D loss: 0.692895, acc: 0.500000]  [A loss: 0.771060, acc: 0.132812]
338: [D loss: 0.691781, acc: 0.511719]  [A loss: 0.739587, acc: 0.226562]
339: [D loss: 0.696446, acc: 0.505859]  [A loss: 0.799424, acc: 0.070312]
340: [D loss: 0.683302, acc: 0.580078]  [A loss: 0.690472, acc: 0.546875]
341: [D loss: 0.741591, acc: 0.492188]  [A loss: 0.897553, acc: 0.019531]
342: [D loss: 0.692054, acc: 0.539062]  [A loss: 0.736870, acc: 0.339844]
343: [D loss: 0.725621, acc: 0.468750]  [A loss: 0.839577, acc: 0.015625]
344: [D loss: 0.689288, acc: 0.535156]  [A loss: 0.710510, acc: 0.398438]
345: [D loss: 0.706692, acc: 0.501953]  [A loss: 0.771405, acc: 0.117188]
346: [D loss: 0.691194, acc: 0.531250]  [A loss: 0.729892, acc: 0.312500]
347: [D loss: 0.696210, acc: 0.507812]  [A loss: 0.758395, acc: 0.179688]
348: [D loss: 0.696911, acc: 0.494141]

...

9471: [D loss: 0.699704, acc: 0.507812]  [A loss: 0.774835, acc: 0.246094]
9472: [D loss: 0.702656, acc: 0.523438]  [A loss: 0.790548, acc: 0.261719]
9473: [D loss: 0.711687, acc: 0.476562]  [A loss: 0.759228, acc: 0.308594]
9474: [D loss: 0.708843, acc: 0.503906]  [A loss: 0.769401, acc: 0.265625]
9475: [D loss: 0.718290, acc: 0.482422]  [A loss: 0.786108, acc: 0.218750]
9476: [D loss: 0.707462, acc: 0.501953]  [A loss: 0.772605, acc: 0.261719]
9477: [D loss: 0.704774, acc: 0.521484]  [A loss: 0.784852, acc: 0.257812]
9478: [D loss: 0.700822, acc: 0.519531]  [A loss: 0.764699, acc: 0.292969]
9479: [D loss: 0.713286, acc: 0.494141]  [A loss: 0.820316, acc: 0.191406]
9480: [D loss: 0.716206, acc: 0.494141]  [A loss: 0.770147, acc: 0.253906]
9481: [D loss: 0.715419, acc: 0.496094]  [A loss: 0.821157, acc: 0.179688]
9482: [D loss: 0.711712, acc: 0.500000]  [A loss: 0.788756, acc: 0.253906]
9483: [D loss: 0.720315, acc: 0.470703]  [A loss: 0.819706, acc: 0.152344]
9484: [D loss: 0.698063, 

9581: [D loss: 0.695269, acc: 0.507812]  [A loss: 0.798782, acc: 0.218750]
9582: [D loss: 0.706225, acc: 0.507812]  [A loss: 1.005013, acc: 0.082031]
9583: [D loss: 0.703686, acc: 0.535156]  [A loss: 0.696995, acc: 0.492188]
9584: [D loss: 0.742497, acc: 0.515625]  [A loss: 1.008247, acc: 0.054688]
9585: [D loss: 0.689182, acc: 0.546875]  [A loss: 0.687502, acc: 0.531250]
9586: [D loss: 0.741812, acc: 0.503906]  [A loss: 1.065385, acc: 0.000000]
9587: [D loss: 0.693930, acc: 0.494141]  [A loss: 0.693807, acc: 0.496094]
9588: [D loss: 0.737498, acc: 0.507812]  [A loss: 0.984340, acc: 0.035156]
9589: [D loss: 0.706253, acc: 0.488281]  [A loss: 0.866555, acc: 0.175781]
9590: [D loss: 0.700988, acc: 0.517578]  [A loss: 0.813201, acc: 0.226562]
9591: [D loss: 0.712713, acc: 0.484375]  [A loss: 0.829431, acc: 0.167969]
9592: [D loss: 0.710892, acc: 0.500000]  [A loss: 0.786624, acc: 0.195312]
9593: [D loss: 0.705587, acc: 0.513672]  [A loss: 0.831954, acc: 0.179688]
9594: [D loss: 0.708925, 

9691: [D loss: 0.713551, acc: 0.505859]  [A loss: 0.613555, acc: 0.734375]
9692: [D loss: 0.749284, acc: 0.496094]  [A loss: 1.215172, acc: 0.000000]
9693: [D loss: 0.704563, acc: 0.511719]  [A loss: 0.593714, acc: 0.781250]
9694: [D loss: 0.831145, acc: 0.503906]  [A loss: 1.246955, acc: 0.019531]
9695: [D loss: 0.717485, acc: 0.521484]  [A loss: 0.245811, acc: 0.984375]
9696: [D loss: 2.312388, acc: 0.500000]  [A loss: 0.940998, acc: 0.062500]
9697: [D loss: 0.716249, acc: 0.488281]  [A loss: 0.793422, acc: 0.238281]
9698: [D loss: 0.750873, acc: 0.488281]  [A loss: 0.803451, acc: 0.234375]
9699: [D loss: 0.723624, acc: 0.458984]  [A loss: 0.721573, acc: 0.410156]
9700: [D loss: 0.742195, acc: 0.482422]  [A loss: 0.790247, acc: 0.238281]
9701: [D loss: 0.741898, acc: 0.458984]  [A loss: 0.757365, acc: 0.300781]
9702: [D loss: 0.740120, acc: 0.476562]  [A loss: 0.737646, acc: 0.375000]
9703: [D loss: 0.733320, acc: 0.468750]  [A loss: 0.751295, acc: 0.324219]
9704: [D loss: 0.722454, 

9801: [D loss: 0.690350, acc: 0.544922]  [A loss: 2.658327, acc: 0.007812]
9802: [D loss: 0.737813, acc: 0.505859]  [A loss: 0.450717, acc: 0.968750]
9803: [D loss: 0.836662, acc: 0.500000]  [A loss: 0.957122, acc: 0.031250]
9804: [D loss: 0.688150, acc: 0.533203]  [A loss: 0.694173, acc: 0.531250]
9805: [D loss: 0.760901, acc: 0.492188]  [A loss: 1.053519, acc: 0.015625]
9806: [D loss: 0.697821, acc: 0.509766]  [A loss: 0.704848, acc: 0.472656]
9807: [D loss: 0.736276, acc: 0.517578]  [A loss: 0.944856, acc: 0.031250]
9808: [D loss: 0.704577, acc: 0.519531]  [A loss: 0.721124, acc: 0.421875]
9809: [D loss: 0.717318, acc: 0.511719]  [A loss: 0.794632, acc: 0.218750]
9810: [D loss: 0.708870, acc: 0.492188]  [A loss: 0.744234, acc: 0.355469]
9811: [D loss: 0.705963, acc: 0.511719]  [A loss: 0.778857, acc: 0.273438]
9812: [D loss: 0.699934, acc: 0.517578]  [A loss: 0.788993, acc: 0.238281]
9813: [D loss: 0.707107, acc: 0.501953]  [A loss: 0.778302, acc: 0.296875]
9814: [D loss: 0.700779, 

9911: [D loss: 0.750936, acc: 0.494141]  [A loss: 0.661007, acc: 0.613281]
9912: [D loss: 0.751040, acc: 0.500000]  [A loss: 0.710662, acc: 0.464844]
9913: [D loss: 0.734469, acc: 0.490234]  [A loss: 0.729001, acc: 0.398438]
9914: [D loss: 0.759085, acc: 0.500000]  [A loss: 0.684174, acc: 0.562500]
9915: [D loss: 0.754596, acc: 0.492188]  [A loss: 0.705195, acc: 0.519531]
9916: [D loss: 0.726163, acc: 0.505859]  [A loss: 0.708196, acc: 0.468750]
9917: [D loss: 0.736130, acc: 0.503906]  [A loss: 0.761008, acc: 0.367188]
9918: [D loss: 0.746464, acc: 0.498047]  [A loss: 0.701705, acc: 0.484375]
9919: [D loss: 0.742082, acc: 0.492188]  [A loss: 0.729190, acc: 0.464844]
9920: [D loss: 0.724116, acc: 0.500000]  [A loss: 0.731179, acc: 0.359375]
9921: [D loss: 0.735742, acc: 0.498047]  [A loss: 0.718589, acc: 0.445312]
9922: [D loss: 0.729621, acc: 0.503906]  [A loss: 0.741323, acc: 0.355469]
9923: [D loss: 0.740462, acc: 0.484375]  [A loss: 0.716220, acc: 0.453125]
9924: [D loss: 0.734757, 